### Parses all champ stats from a hosted server
This allows us to retrieve descriptions for all abilities and retrieve ability related info.

In [1]:
import os

CHAMP_STATS_URL = "https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/champions/"

In [2]:
import requests
from bs4 import BeautifulSoup

page = requests.get(CHAMP_STATS_URL)
soup = BeautifulSoup(page.content, "html.parser")
champ_hrefs = soup.find_all(href=lambda x: "json" in x if x else False)
champ_jsons = [champ_href.contents[0] for champ_href in champ_hrefs]
champ_json_urls = [f"{CHAMP_STATS_URL}{champ_json}" for champ_json in champ_jsons]

In [3]:
import urllib.request, json

character_abilities = {}
for champ_json_url in champ_json_urls:
    with urllib.request.urlopen(champ_json_url) as url:
        data = json.load(url)
        name = data['name']
        abilities = data['abilities']
        character_abilities[name] = abilities

KeyboardInterrupt: 

In [5]:
with open("characters.json", "w+") as f:
    json.dump(character_abilities, f)

### Video scraper scripts
Want to download all the videos that are in the LoL wiki.

In [8]:
from bs4 import BeautifulSoup
import requests
import os

LOL_WIKI = "https://www.leagueoflegends.com/en-us/"
LIST_OF_CHAMPS = LOL_WIKI + "/champions/"

champs_soup = BeautifulSoup(requests.get(url=LIST_OF_CHAMPS).content, 'html.parser')

In [30]:
link_tags = champs_soup.find_all('a')
champ_links = [LOL_WIKI[:-1] + link['href'] for link in link_tags]
len(champ_links)

169

In [28]:
import re
import os
import requests

VIDEO_PATTERN = r"_([PQWER]\d+)\.webm"
ABILITY_TYPES = ["P", "Q", "W", "E", "R"]
CHUNK_SIZE = 1024  # Size of chunks for streaming download

def download_video(url, save_path):
    try:
        download_req = requests.get(url, stream=True, timeout=10)
        if download_req.status_code != 200:
            print(f"Failed to download: {url} (Status code: {download_req.status_code})")
            return False

        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        with open(save_path, 'wb') as file:
            for chunk in download_req.iter_content(chunk_size=CHUNK_SIZE):
                if chunk:  # Filter out keep-alive chunks
                    file.write(chunk)

        print(f"Downloaded: {save_path}")
        return True
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")
        return False

def generate_urls_and_download(base_url, download_path):
    match = re.search(VIDEO_PATTERN, base_url)
    if not match:
        print("No match found in the URL")
        return

    base, ability_part = match.span(1)
    for key in ABILITY_TYPES:
        for i in range(1, 5):  # Adjust the range as needed
            new_url = f"{base_url[:base]}{key}{i}.webm"
            file_name = f"{key}{i}.webm"
            save_path = os.path.join(download_path, file_name)

            # Attempt to download the video
            if not download_video(new_url, save_path):
                print(f"Stopping further attempts for {key} abilities at index {i}")
                break

In [4]:
import re

SAVE_DIR = "Data/Videos"
os.makedirs(SAVE_DIR, exist_ok=True)

for url in champ_links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    video_url = soup.find(attrs={"data-testid": "source-video/webm"})['src']
    
    champ_name = soup.find('div', attrs={'data-testid': 'title'}).text
    if os.path.exists(f"{SAVE_DIR}/{champ_name}"):
        print(f"Videos already downloaded for champ: {champ_name}")
        continue
        
    generate_urls_and_download(video_url, f"{SAVE_DIR}/{champ_name}")

NameError: name 'champ_links' is not defined

In [6]:
import os
from moviepy.video.io.VideoFileClip import VideoFileClip

def get_video_durations(directory):
    durations = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.webm'):
                video_path = os.path.join(root, file)
                try:
                    with VideoFileClip(video_path) as video:
                        durations.append(video.duration)  # Duration in seconds
                except Exception as e:
                    print(f"Error processing {video_path}: {e}")
    return durations

In [7]:
import numpy as np

video_durations = get_video_durations(SAVE_DIR)
np.mean(video_durations)

Error processing Data/Videos/Pantheon/W1.webm: MoviePy error: failed to read the duration of file Data/Videos/Pantheon/W1.webm.
Here are the file infos returned by ffmpeg:

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1732155217949/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/mi

np.float64(8.198829787234043)

### Saving Champ Info with the champ
Now we can save all data together with the videos in a clean way

In [8]:
with open("LoLChampInfo.json") as f:
    champ_data = json.load(f)    

In [9]:
for champ_dir in os.listdir(SAVE_DIR):
    champ_stats = champ_data.get(champ_dir)
    champ_path = os.path.join(SAVE_DIR, champ_dir)
    
    if champ_stats:
        stats_file_path = os.path.join(champ_path, "stats.json")
        with open(stats_file_path, 'w') as stats_file:
            json.dump(champ_stats, stats_file, indent=4)
        print(f"Stats for {champ_dir} saved to {stats_file_path}")
    else:
        print(f"No stats found for {champ_dir}")

Stats for Rakan saved to Data/Videos/Rakan/stats.json
Stats for Smolder saved to Data/Videos/Smolder/stats.json
Stats for Diana saved to Data/Videos/Diana/stats.json
Stats for Vayne saved to Data/Videos/Vayne/stats.json
Stats for Nilah saved to Data/Videos/Nilah/stats.json
Stats for Sion saved to Data/Videos/Sion/stats.json
Stats for K'Sante saved to Data/Videos/K'Sante/stats.json
Stats for Master Yi saved to Data/Videos/Master Yi/stats.json
Stats for Zed saved to Data/Videos/Zed/stats.json
Stats for Talon saved to Data/Videos/Talon/stats.json
Stats for Akshan saved to Data/Videos/Akshan/stats.json
Stats for Zoe saved to Data/Videos/Zoe/stats.json
Stats for Akali saved to Data/Videos/Akali/stats.json
Stats for Bard saved to Data/Videos/Bard/stats.json
Stats for Riven saved to Data/Videos/Riven/stats.json
Stats for Vi saved to Data/Videos/Vi/stats.json
Stats for Bel'Veth saved to Data/Videos/Bel'Veth/stats.json
Stats for Ashe saved to Data/Videos/Ashe/stats.json
Stats for Syndra saved t

In [43]:
import os
import json

VISUAL_KEYS = {
    'name',
    'effects',
    'targeting',
    'affects',
    'spellEffects',
    'projectile',
    'speed',
    'width',
    'castTime',
    'effectRadius',
    'notes',
    'angle'
}

count = 0

def extract_visual_data(data):
    if 'effects' in data and isinstance(data['effects'], list):
        data['effects'] = ' '.join([effect.get('description', '') for effect in data['effects']])

    visual_data = {key: value for key, value in data.items() if key in VISUAL_KEYS and value is not None}
    return visual_data

def process_champion_abilities(champ_dir, stats_file_path):
    global count
    with open(stats_file_path, 'r') as stats_file:
        champ_data = json.load(stats_file)

    for ability_key in ['P', 'Q', 'W', 'E', 'R']:
        ability_data = champ_data.get(ability_key)
        for i, ability in enumerate(ability_data):
            count += 1
            visual_data = extract_visual_data(ability)
            ability_file_path = os.path.join(champ_dir, f"{ability_key}{i+1}.json")
            with open(ability_file_path, 'w') as ability_file:
                json.dump(visual_data, ability_file, indent=4)
            print(f"Saved {ability_key}{i+1}.json in {champ_dir}")

In [44]:
for champ_dir in os.listdir(SAVE_DIR):
    champ_path = os.path.join(SAVE_DIR, champ_dir)
    stats_file_path = os.path.join(champ_path, "stats.json")

    if os.path.isdir(champ_path) and os.path.exists(stats_file_path):
        print(f"Processing {champ_dir}...")
        process_champion_abilities(champ_path, stats_file_path)
    else:
        print(f"No stats.json found for {champ_dir}")

Processing Rakan...
Saved P1.json in Data/Videos/Rakan
Saved Q1.json in Data/Videos/Rakan
Saved W1.json in Data/Videos/Rakan
Saved E1.json in Data/Videos/Rakan
Saved R1.json in Data/Videos/Rakan
Processing Smolder...
Saved P1.json in Data/Videos/Smolder
Saved Q1.json in Data/Videos/Smolder
Saved W1.json in Data/Videos/Smolder
Saved E1.json in Data/Videos/Smolder
Saved R1.json in Data/Videos/Smolder
Processing Diana...
Saved P1.json in Data/Videos/Diana
Saved Q1.json in Data/Videos/Diana
Saved W1.json in Data/Videos/Diana
Saved E1.json in Data/Videos/Diana
Saved R1.json in Data/Videos/Diana
Processing Vayne...
Saved P1.json in Data/Videos/Vayne
Saved Q1.json in Data/Videos/Vayne
Saved W1.json in Data/Videos/Vayne
Saved E1.json in Data/Videos/Vayne
Saved R1.json in Data/Videos/Vayne
Processing Nilah...
Saved P1.json in Data/Videos/Nilah
Saved Q1.json in Data/Videos/Nilah
Saved W1.json in Data/Videos/Nilah
Saved E1.json in Data/Videos/Nilah
Saved R1.json in Data/Videos/Nilah
Processing Si

In [45]:
count

909